# Table of Content

# Introduction

## Setup and Import

As always, the first step is to import the required libraries and data. Since we do not want to run the SQL query every time, we can simply import the csv file we created in the first notebook.

In [1]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import altair as alt
import numpy as np

from ipywidgets import HTML
from io import BytesIO
import base64

import warnings
warnings.simplefilter("ignore")

# Turn off the max column width so the images won't be truncated
pd.set_option('display.max_colwidth', None)
# Show all Columns
pd.set_option('display.max_columns', None)
pd. set_option('display.max_rows', None)
 
# Turning off the max column will display all the data
# if gathering into sets / array we might want to restrict to a few items
pd.set_option('display.max_seq_items', 50)
pd.set_option('display.width', 1000)


In [2]:
# Import dataframes for Stock Prices
stock_price_df = pd.read_csv('../data/train_files/stock_prices.csv', parse_dates=['Date'])
sec_df = pd.read_csv('../data/train_files/secondary_stock_prices.csv', parse_dates=['Date'])
tra_df = pd.read_csv('../data/train_files/trades.csv', parse_dates=['Date'])

stock_desc_df = pd.read_csv('../data/stock_price_spec.csv')
stock_list_desc_df = pd.read_csv('../data/stock_list_spec.csv')
stock_list = pd.read_csv('../data/stock_list.csv')

In [3]:
stock_price_df.head(5)

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [4]:
stock_list.head()

,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close,IssuedShares,MarketCapitalization,Universe0
0,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,1.092828e+07,3.365911e+10,True
1,1305,20211230,Daiwa ETF-TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2097.0,3.634636e+09,7.621831e+12,False
2,1306,20211230,NEXT FUNDS TOPIX Exchange Traded Fund,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2073.5,7.917718e+09,1.641739e+13,False
3,1308,20211230,Nikko Exchange Traded Index Fund TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2053.0,3.736943e+09,7.671945e+12,False
4,1309,20211230,NEXT FUNDS ChinaAMC SSE50 Index Exchange Traded Fund,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,44280.0,7.263200e+04,3.216145e+09,False


In [5]:
stock_desc_df 


,Column,Sample value,Type,Addendum,Remarks
0,RowId,20170104_1301,string,NaN,Unique ID of price records
1,Date,2017-01-04 0:00:00,date,NaN,Trade date
2,SecuritiesCode,1301,Int64,NaN,Local securities code
3,Open,2734,float,NaN,first traded price on a day
4,High,2755,float,NaN,highest traded price on a day
5,Low,2730,float,NaN,lowest traded price on a day
6,Close,2742,float,NaN,last traded price on a day
7,Volume,31400,Int64,NaN,number of traded stocks on a day
8,AdjustmentFactor,1,float,NaN,to calculate theoretical price/volume when split/reverse-split happens (NOT including dividend/allotment of shares/)
9,SupervisionFlag,FALSE,boolean,NaN,Flag of Securities Under Supervision & Securities to Be Delisted\nhttps://www.jpx.co.jp/english/listing/market-alerts/supervision/00-archives/index.html )


In [6]:
stock_list_desc_df

,Column,Sample value,Type,Addendum,Remarks
0,SecuritiesCode,1301,Int64,NaN,Local Securities Code
1,EffectiveDate,20211230,date,NaN,the effective date
2,Name,"KYOKUYO CO.,LTD.",string,NaN,Name of security
3,Section/Products,First Section (Domestic),string,NaN,Section/Product
4,NewMarketSegment,Prime Market,string,NaN,New market segment effective from 2022-04-04 (as of 15:30 JST on Mar 11 2022)\nref. https://www.jpx.co.jp/english/equities/market-restructure/market-segments/index.html
5,33SectorCode,50,Int64,NaN,33 Sector Name\n\nref. https://www.jpx.co.jp/english/markets/indices/line-up/files/e_fac_13_sector.pdf
6,33SectorName,"Fishery, Agriculture and Forestry",string,NaN,33 Sector Name\n\nref. https://www.jpx.co.jp/english/markets/indices/line-up/files/e_fac_13_sector.pdf
7,17SectorCode,1,Int64,NaN,17 Sector Code\nref. https://www.jpx.co.jp/english/markets/indices/line-up/files/e_fac_13_sector.pdf
8,17SectorName,FOODS,string,NaN,17 Sector Name\nref. https://www.jpx.co.jp/english/markets/indices/line-up/files/e_fac_13_sector.pdf
9,NewIndexSeriesSizeCode,7,Int64,NaN,TOPIX New Index Series code\n\nref. https://www.jpx.co.jp/english/markets/indices/line-up/files/e_fac_12_size.pdf


## Additional Features

### Metrics
BOP: Balance of Power = (Close price – Open price) / (High price – Low price) <br>
av: average = (Close price + High price + Low price) / 3 <br>
vwap: Volume-weighted average price = (Close price * Volume) / Volume <br>
HLr: High-Low-Range = High price – Low price <br>
OCr: Open-Close-Range = Close price – Open price <br>
OC: Open-Close = Close price * Open price <br>
relC: relative Change = (Close price – Open price)/Open Price <br>
HL: High-Low = High price * Low price <br>
logC: logarithmic scaled Close price <br>
logR: log(Close price) - log(Open price) <br>
OHLCstd: Open, High, Low, Close, standard deviated. <br>
OHLCskew: Open, High, Low, Close, skewed. <br>
OHLCkur: Open, High, Low, Close, kurtosis. <br>
Cpos: = (Close price – Low price) / (High price – Low price) -0.5 <br>
Opos: = (Open price – Low price) / (High price – Low price) -0.5 <br>
bsforce: = Cpos * Volume <br>
    
    
### Weekdays
Weekday = Day of the Week (1-5) <br>
Monday = if Monday == 1 <br>
Tuesday = if Tuesday == 1 <br>
Wednesday = if Wednesday == 1 <br>
Thursday = if Thursday == 1 <br>
Friday = if Fryday == 1 <br>
Date = Date

In [7]:
def FE(stock_price_df):
    stock_price_df['BOP'] = (stock_price_df['Open']-stock_price_df['Close'])/(stock_price_df['High']-stock_price_df['Low'])
    stock_price_df['Cpos'] = (stock_price_df['Close']-stock_price_df['Low'])/(stock_price_df['High']-stock_price_df['Low']) -0.5
    stock_price_df['bsforce'] = stock_price_df['Cpos'] * stock_price_df['Volume']
    stock_price_df['Opos'] = (stock_price_df['Open']-stock_price_df['Low'])/(stock_price_df['High']-stock_price_df['Low']) -0.5


    
    stock_price_df['av'] = (stock_price_df['Close']+stock_price_df['High']+stock_price_df['Low'])/3
    stock_price_df['vwap'] = (stock_price_df['Close'] * stock_price_df['Volume'])/ stock_price_df['Volume']
    stock_price_df['HLr'] = stock_price_df['High'] - stock_price_df['Low']
    stock_price_df['OCr'] = stock_price_df['Open'] - stock_price_df['Close']
    stock_price_df['relC'] = (stock_price_df['Close'] - stock_price_df['Open'])/stock_price_df['Open']
    stock_price_df['OC'] = stock_price_df['Open'] * stock_price_df['Close']
    stock_price_df['HL'] = stock_price_df['High'] * stock_price_df['Low']
    stock_price_df['logC'] = np.log(stock_price_df['Close']+1)
    stock_price_df['logR'] = np.log(stock_price_df['Close'])-np.log(stock_price_df['Open'])
    stock_price_df['OHLCstd'] = stock_price_df[['Open','High','Low','Close']].std(axis=1)
    stock_price_df['OHLCskew'] = stock_price_df[['Open','High','Low','Close']].skew(axis=1)
    stock_price_df['OHLCkur'] = stock_price_df[['Open','High','Low','Close']].kurtosis(axis=1)


    
    stock_price_df['weekday'] = stock_price_df['Date'].dt.weekday+1
    stock_price_df['Monday'] = np.where(stock_price_df['weekday']==1,1,0)
    stock_price_df['Tuesday'] = np.where(stock_price_df['weekday']==2,1,0)
    stock_price_df['Wednesday'] = np.where(stock_price_df['weekday']==3,1,0)
    stock_price_df['Thursday'] = np.where(stock_price_df['weekday']==4,1,0)
    stock_price_df['Friday'] = np.where(stock_price_df['weekday']==5,1,0)
    stock_price_df['Date'] = pd.to_datetime(stock_price_df['Date'])

    #rolling mean
    stock_price_df['SMA5'] = stock_price_df.Close.rolling(5).mean()
    stock_price_df['SMA20'] = stock_price_df.Close.rolling(20).mean()
    stock_price_df['SMA50'] = stock_price_df.Close.rolling(50).mean()
    stock_price_df['SMA200'] = stock_price_df.Close.rolling(200).mean()
    stock_price_df['SMA500'] = stock_price_df.Close.rolling(500).mean()

    #Exponentially Weighted average
    stock_price_df['EMA5'] = stock_price_df.Close.ewm(span=5, adjust=False).mean()
    stock_price_df['EMA20'] = stock_price_df.Close.ewm(span=20, adjust=False).mean()
    stock_price_df['EMA50'] = stock_price_df.Close.ewm(span=50, adjust=False).mean()
    stock_price_df['EMA200'] = stock_price_df.Close.ewm(span=200, adjust=False).mean()
    stock_price_df['EMA500'] = stock_price_df.Close.ewm(span=500, adjust=False).mean()

    

    return stock_price_df
stock_price_df = FE(stock_price_df)
stock_price_df = pd.merge(stock_price_df,stock_list, on='SecuritiesCode')

In [8]:
stock_price_df.head(10)

,RowId,Date,SecuritiesCode,Open,High,Low,Close_x,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,BOP,av,vwap,HLr,OCr,relC,OC,HL,logC,logR,OHLCstd,OHLCskew,OHLCkur,Cpos,bsforce,Opos,weekday,Monday,Tuesday,Wednesday,Thursday,Friday,SMA5,SMA20,SMA50,SMA200,SMA500,EMA5,EMA20,EMA50,EMA200,EMA500,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close_y,IssuedShares,MarketCapitalization,Universe0
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,-0.320000,2742.333333,2742.0,25.0,-8.0,0.002926,7496628.0,7521150.0,7.916807,0.002922,11.026483,0.941530,0.008495,-0.020000,-628.000000,-0.340000,3,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,2742.000000,2742.000000,2742.000000,2742.000000,2742.000000,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
1,20170105_1301,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.002920,0.416667,2740.000000,2738.0,12.0,5.0,-0.001823,7510334.0,7513045.0,7.915348,-0.001824,5.315073,0.198134,-2.215052,-0.250000,-4475.000000,0.166667,4,0,0,0,1,0,1634.4,4667.40,3253.22,NaN,NaN,2469.865742,4128.913255,3611.146767,2591.486142,2244.789563,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
2,20170106_1301,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,False,-0.001092,-0.250000,2734.666667,2740.0,24.0,-6.0,0.002195,7491160.0,7463680.0,7.916078,0.002192,10.503968,-1.164860,1.085094,0.333333,6633.333333,0.083333,5,0,0,0,0,1,1646.2,4653.20,3252.34,2236.6775,2038.7184,2478.198372,4120.312180,3608.506609,2593.691422,2246.059723,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
3,20170110_1301,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,False,-0.005100,-0.157895,2745.666667,2748.0,19.0,-3.0,0.001093,7543260.0,7532190.0,7.918992,0.001092,7.937254,-0.703934,1.125220,0.184211,4457.894737,0.026316,2,0,1,0,0,0,1654.0,4516.45,3201.24,2231.0465,2037.7990,2467.890060,4024.784152,3538.924414,2575.661754,2240.805293,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
4,20170111_1301,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,False,-0.003295,0.200000,2744.666667,2745.0,15.0,3.0,-0.001092,7543260.0,7532224.0,7.917901,-0.001092,6.350853,-0.843252,0.933953,0.033333,310.000000,0.233333,3,0,0,1,0,0,1662.8,NaN,NaN,NaN,NaN,2464.515692,4017.194394,3530.895281,2555.202645,2224.454374,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
5,20170112_1301,2017-01-12,1301,2745.0,2747.0,2703.0,2731.0,28700,1.0,NaN,False,-0.006613,0.318182,2727.000000,2731.0,44.0,14.0,-0.005100,7496595.0,7425141.0,7.912789,-0.005113,20.289570,-1.354079,1.265400,0.136364,3913.636364,0.454545,4,0,0,0,1,0,1654.0,NaN,NaN,NaN,NaN,2458.791049,4016.507971,3530.752970,2564.766409,2233.147617,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
6,20170113_1301,2017-01-13,1301,2707.0,2730.0,2707.0,2722.0,19400,1.0,NaN,False,-0.006657,-0.652174,2719.666667,2722.0,23.0,-15.0,0.005541,7368454.0,7390110.0,7.909489,0.005526,11.445523,0.405505,-3.706427,0.152174,2952.173913,-0.500000,5,0,0,0,0,1,1640.0,4398.10,NaN,NaN,NaN,2425.478169,3935.117134,3474.914378,2536.401311,2211.646207,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,

In [9]:
df = stock_price_df.drop(columns=['17SectorCode','NewIndexSeriesSizeCode', '33SectorCode', 'ExpectedDividend', 'RowId', 'Close_y'])

In [10]:
df.shape

(2332531, 53)

In [11]:
df.rename(columns={"Close_x": "Close"}, inplace=True)

In [12]:
df.head()

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,SupervisionFlag,Target,BOP,av,vwap,HLr,OCr,relC,OC,HL,logC,logR,OHLCstd,OHLCskew,OHLCkur,Cpos,bsforce,Opos,weekday,Monday,Tuesday,Wednesday,Thursday,Friday,SMA5,SMA20,SMA50,SMA200,SMA500,EMA5,EMA20,EMA50,EMA200,EMA500,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorName,17SectorName,NewIndexSeriesSize,TradeDate,IssuedShares,MarketCapitalization,Universe0
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,False,0.000730,-0.320000,2742.333333,2742.0,25.0,-8.0,0.002926,7496628.0,7521150.0,7.916807,0.002922,11.026483,0.941530,0.008495,-0.020000,-628.000000,-0.340000,3,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,2742.000000,2742.000000,2742.000000,2742.000000,2742.000000,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True
1,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,False,0.002920,0.416667,2740.000000,2738.0,12.0,5.0,-0.001823,7510334.0,7513045.0,7.915348,-0.001824,5.315073,0.198134,-2.215052,-0.250000,-4475.000000,0.166667,4,0,0,0,1,0,1634.4,4667.40,3253.22,NaN,NaN,2469.865742,4128.913255,3611.146767,2591.486142,2244.789563,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True
2,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,False,-0.001092,-0.250000,2734.666667,2740.0,24.0,-6.0,0.002195,7491160.0,7463680.0,7.916078,0.002192,10.503968,-1.164860,1.085094,0.333333,6633.333333,0.083333,5,0,0,0,0,1,1646.2,4653.20,3252.34,2236.6775,2038.7184,2478.198372,4120.312180,3608.506609,2593.691422,2246.059723,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True
3,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,False,-0.005100,-0.157895,2745.666667,2748.0,19.0,-3.0,0.001093,7543260.0,7532190.0,7.918992,0.001092,7.937254,-0.703934,1.125220,0.184211,4457.894737,0.026316,2,0,1,0,0,0,1654.0,4516.45,3201.24,2231.0465,2037.7990,2467.890060,4024.784152,3538.924414,2575.661754,2240.805293,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True
4,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,False,-0.003295,0.200000,2744.666667,2745.0,15.0,3.0,-0.001092,7543260.0,7532224.0,7.917901,-0.001092,6.350853,-0.843252,0.933953,0.033333,310.000000,0.233333,3,0,0,1,0,0,1662.8,NaN,NaN,NaN,NaN,2464.515692,4017.194394,3530.895281,2555.202645,2224.454374,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True


In [13]:
df.shape

(2332531, 53)

### Additional features with ta library

In [14]:
import ta as ta
from ta import add_all_ta_features
from ta.utils import dropna

In [15]:
df = ta.add_all_ta_features(df, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=False)

In [16]:
df.sample(10)

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,SupervisionFlag,Target,BOP,av,vwap,HLr,OCr,relC,OC,HL,logC,logR,OHLCstd,OHLCskew,OHLCkur,Cpos,bsforce,Opos,weekday,Monday,Tuesday,Wednesday,Thursday,Friday,SMA5,SMA20,SMA50,SMA200,SMA500,EMA5,EMA20,EMA50,EMA200,EMA500,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorName,17SectorName,NewIndexSeriesSize,TradeDate,IssuedShares,MarketCapitalization,Universe0,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_sma_em,volume_vpt,volume_vwap,volume_mfi,volume_nvi,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_atr,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_stc,trend_adx,trend_adx_pos,trend_adx_neg,trend_cci,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
415607,2020-10-07,3191,1643.0,1654.0,1627.0,1637.0,127500,1.0,False,-0.019370,0.222222,1639.333333,1637.0,27.0,6.0,-0.003652,2689591.0,2691058.0,7.401231,-0.003659,11.295279,0.120222,0.166216,-0.129630,-16527.777778,0.092593,3,0,0,1,0,0,2046.4,1831.900,2414.940,NaN,NaN,1665.542481,1834.525229,2201.261710,2494.188367,2533.715500,20211230,"JOYFUL HONDA CO.,LTD.",First Section (Domestic),Prime Market,Retail Trade,RETAIL TRADE,TOPIX Small 1,20211230.0,73348911.0,1.080429e+11,True,2.051062e+08,12336453600,0.240465,1.586241e+06,-8.576471e+05,NaN,-3938.028529,NaN,NaN,2.082422e+09,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1750.407407,1653.518519,NaN,-0.170489,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.719472,50.656048,-1.936575,NaN,NaN,1658.375405,1609.655933,NaN,NaN,NaN,0.443663,25.940162,NaN,NaN,55.464572,NaN,NaN,NaN,NaN,1590.0,8.260273e+01,NaN,NaN,NaN,NaN,1529.25,1461.5,NaN,NaN,NaN,NaN,1774.772800,0.0,0.0,58.274068,0.000000,0.340636,0.516261,32.219580,NaN,NaN,NaN,NaN,NaN,2.504696,3.026701,3.177745,-0.151044,-5.902385,0.427815,-6.330199,NaN,-1.385542,-1.395230,-40.299052
449252,2020-09-18,3349,17800.0,17860.0,17610.0,17760.0,185700,1.0,False,0.008869,0.160000,17743.333333,17760.0,250.0,40.0,-0.002247,316128000.0,314514600.0,9.784760,-0.002250,106.575482,-1.131952,1.666018,0.100000,18570.000000,0.260000,5,0,0,0,0,1,4767.6,2215.400,1949.760,NaN,NaN,6983.840186,3048.822365,2325.481809,2375.460771,2466.333961,20211230,COSMOS Pharmaceutical Corporation,First Section (Domestic),Prime Market,Retail Trade,RETAIL TRADE,TOPIX Mid400,20211230.0,40000800.0,6.772135e+11,True,2.077364e+08,13025085300,0.037299,-9.880853e+06,6.731287e+05,-1.811936e+07,-1292.618814,17830.795186,53.266033,1.698072e+10,18201.50,19440.068125,16962.931875,13.609517,0.321770,0.0,0.0,17671.666667,18096.666667,17246.666667,4.809959,0.603922,0.0,0.0,17110.0,19740.0,18425.0,14.449359,0.247148,NaN,7.806194,-269.116227,-245.790773,-23.325454,17808.333333,18396.923077,17912.282473,18181.398700,0.860738,1.053691,-0.192953,-0.186618,24.823883,408.50,-60.911582,-37.092474,-23.819108,17655.0,18425.0,18040.00,18380.0,1.338983e+01,NaN,NaN,NaN,-57.699927,18690.00,17660.0,24.0,64.0,-40.0,16354.273858,NaN,0.0,0.0,43.882395,0.704557,0.782074,0.663585,-13.240487,57.014230,38.235294,43.529412,-61.7

In [17]:
df.shape

(2332531, 139)

In [18]:
df.isnull().sum() * 100 / len(df)

Date                          0.000000
SecuritiesCode                0.000000
Open                          0.326169
High                          0.326169
Low                           0.326169
Close                         0.326169
Volume                        0.000000
AdjustmentFactor              0.000000
SupervisionFlag               0.000000
Target                        0.010204
BOP                           0.747943
av                            0.326169
vwap                          0.326169
HLr                           0.326169
OCr                           0.326169
relC                          0.326169
OC                            0.326169
HL                            0.326169
logC                          0.326169
logR                          0.326169
OHLCstd                       0.326169
OHLCskew                      0.326169
OHLCkur                       0.326169
Cpos                          0.747943
bsforce                       0.747943
Opos                     

In [19]:
df = df.drop(columns=['momentum_kama','trend_adx', 'trend_adx_pos', 'trend_adx_pos', 'volatility_atr'])

In [20]:
df.head()

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,SupervisionFlag,Target,BOP,av,vwap,HLr,OCr,relC,OC,HL,logC,logR,OHLCstd,OHLCskew,OHLCkur,Cpos,bsforce,Opos,weekday,Monday,Tuesday,Wednesday,Thursday,Friday,SMA5,SMA20,SMA50,SMA200,SMA500,EMA5,EMA20,EMA50,EMA200,EMA500,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorName,17SectorName,NewIndexSeriesSize,TradeDate,IssuedShares,MarketCapitalization,Universe0,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_sma_em,volume_vpt,volume_vwap,volume_mfi,volume_nvi,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_stc,trend_adx_neg,trend_cci,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,others_dr,others_dlr,others_cr
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,False,0.000730,-0.320000,2742.333333,2742.0,25.0,-8.0,0.002926,7496628.0,7521150.0,7.916807,0.002922,11.026483,0.941530,0.008495,-0.020000,-628.000000,-0.340000,3,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,2742.000000,2742.000000,2742.000000,2742.000000,2742.000000,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True,-1256.000000,31400,NaN,NaN,NaN,NaN,3498.092268,NaN,NaN,1000.000000,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,2767.333333,2717.333333,NaN,0.493333,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2742.5,NaN,0.0,NaN,2575.798578,2618.177553,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.704519,NaN,0.000000
1,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,False,0.002920,0.416667,2740.000000,2738.0,12.0,5.0,-0.001823,7510334.0,7513045.0,7.915348,-0.001824,5.315073,0.198134,-2.215052,-0.250000,-4475.000000,0.166667,4,0,0,0,1,0,1634.4,4667.40,3253.22,NaN,NaN,2469.865742,4128.913255,3611.146767,2591.486142,2244.789563,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True,-10206.000000,13500,NaN,NaN,-1.005587e+05,NaN,1765.106557,NaN,NaN,998.541211,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,2759.666667,2722.666667,NaN,0.414414,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2742.5,NaN,0.0,NaN,2575.798578,2618.177553,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.145879,-0.145985,-0.145879
2,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,False,-0.001092,-0.250000,2734.666667,2740.0,24.0,-6.0,0.002195,7491160.0,7463680.0,7.916078,0.002192,10.503968,-1.164860,1.085094,0.333333,6633.333333,0.083333,5,0,0,0,0,1,1646.2,4653.20,3252.34,2236.6775,2038.7184,2478.198372,4120.312180,3608.506609,2593.691422,2246.059723,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True,3060.666667,33400,NaN,NaN,-1.085427e+06

In [21]:
df.shape

(2332531, 135)

In [22]:
df = df[~((df['Open'].isna()) & (df['High'].isna()) & (df['Low'].isna()) & (df['Close'].isna()))]

In [23]:
df.shape

(2324923, 135)

In [24]:
df.isnull().sum() * 100 / len(df)

Date                          0.000000
SecuritiesCode                0.000000
Open                          0.000000
High                          0.000000
Low                           0.000000
Close                         0.000000
Volume                        0.000000
AdjustmentFactor              0.000000
SupervisionFlag               0.000000
Target                        0.000000
BOP                           0.423154
av                            0.000000
vwap                          0.000000
HLr                           0.000000
OCr                           0.000000
relC                          0.000000
OC                            0.000000
HL                            0.000000
logC                          0.000000
logR                          0.000000
OHLCstd                       0.000000
OHLCskew                      0.000000
OHLCkur                       0.000000
Cpos                          0.423154
bsforce                       0.423154
Opos                     

## Notes
### Potential Features

- Earthquakes?